In [1]:
import pandas as pd
import multiprocessing
import time
import numpy as np
import csv
import sys
from itertools import chain

# Aumentar el límite del campo CSV, esencial para archivos grandes
csv.field_size_limit(sys.maxsize)

# Solución al SettingWithCopyWarning: Desactivamos la advertencia para que no interfiera
# en la salida, ya que la lógica en process_chunk ya usa una copia segura.
pd.options.mode.chained_assignment = None

# --- ALGORITMOS DE ORDENAMIENTO Y DEPURACIÓN ---

def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        # Criterio de ordenamiento: de mayor a menor temperatura (Índice 2)
        if left[i][2] >= right[j][2]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result

# Quick Sort-style para depurar: filtra valores no numéricos o 0.0
def depurar_y_particionar(arr):
    n = len(arr)
    i = -1
    for j in range(n):
        temp_val = arr[j][2]
        if isinstance(temp_val, (int, float)) and temp_val != 0.0:
            i += 1
            arr[i], arr[j] = arr[j], arr[i]
    return i + 1

# --- LÓGICA DE PROCESAMIENTO (CON MULTIPROCESAMIENTO) ---

def process_chunk(chunk):
    
    #Realiza la limpieza a nivel de fila de forma paralela.
    
    if chunk.empty:
        return pd.DataFrame()

    try:
        # Solución al SettingWithCopyWarning: Trabajamos con una copia explícita para evitar la advertencia
        chunk = chunk.copy()

        # 1. Depuración Nivel 1 (Guiones y Cadenas a NaN)
        chunk['AvgTemperature'] = chunk['AvgTemperature'].replace(r'["\(\)-]', '', regex=True)
        chunk['AvgTemperature'] = pd.to_numeric(chunk['AvgTemperature'], errors='coerce')

        # 2. Limpieza de fecha
        chunk['Fecha'] = chunk['Fecha'].replace(r'["\(\)]', '', regex=True)

        # 3. Eliminar filas con NaN en temperatura o fecha
        chunk = chunk.dropna(subset=['AvgTemperature', 'Fecha'])

        # 4. Extraer el año
        chunk['Year'] = chunk['Fecha'].str.split('/').str[2]
        chunk = chunk.dropna(subset=['Year'])

        # 5. Filtrado de temperaturas extremas
        chunk = chunk[chunk['AvgTemperature'] > -90]

        # 6. Devolver las columnas necesarias para el ordenamiento
        return chunk[['City', 'Year', 'AvgTemperature']]

    except Exception as e:
        return pd.DataFrame()


def parallel_processing_with_chunks(chunks_to_process, func):
    
    #Divide y mapea la función 'func' a los chunks en paralelo.
    
    num_cores = multiprocessing.cpu_count()
    print(f"\n[PROCESAMIENTO PARALELO] Usando {num_cores} núcleos para {len(chunks_to_process)} bloques.")

    with multiprocessing.Pool(num_cores) as pool:
        results = pool.map(func, chunks_to_process)

    return results

# --- FUNCIÓN PRINCIPAL (MAIN) ---

def main():
    tiempo_INICIAL = time.time()
    archivo_entrada = 'temperaturas.csv'
    archivo_salida = 'procesamiento_paralelo_python_ordenado.csv'
    CHUNK_SIZE = 100000 # Un valor más grande para archivos grandes

    print("\n=======================================================")
    print("Iniciando Procesamiento Paralelo y Ordenamiento de Datos")
    print("=======================================================")

    # 1. LECTURA POR CHUNKS
    print("\n[PASO 1] Leyendo archivo CSV...")
    start_lectura = time.time()
    chunks = []
    try:
        csv_iterator = pd.read_csv(
            archivo_entrada,
            usecols=['City', 'AvgTemperature', 'Fecha'],
            chunksize=CHUNK_SIZE,
            encoding='utf-8',
            low_memory=True,
            skip_blank_lines=True
        )
        for chunk in csv_iterator:
            chunks.append(chunk)

    except Exception as e:
        print(f"\n ERROR: Al leer el archivo {archivo_entrada}. Detalles: {e}")
        return

    end_lectura = time.time()
    total_filas_crudas = sum(len(c) for c in chunks)
    print(f"   Tiempo de lectura: {end_lectura - start_lectura:.4f} segundos")
    print(f"   Filas totales cargadas: {total_filas_crudas:,} registros.")
    print("-------------------------------------------------------")


    # 2. PROCESAMIENTO PARALELO (Limpieza)
    start_procesamiento = time.time()
    results = parallel_processing_with_chunks(chunks, process_chunk)
    valid_results = [res for res in results if not res.empty]

    if not valid_results:
        print("\n Advertencia: No se obtuvieron datos válidos de los procesos paralelos.")
        return

    final_df = pd.concat(valid_results, ignore_index=True)
    end_procesamiento = time.time()
    
    data_list = final_df.to_records(index=False).tolist()
    
    print("-------------------------------------------------------")
    print(f"   Tiempo de limpieza paralelo: {end_procesamiento - start_procesamiento:.4f} segundos")
    print(f"   Registros limpios (listos para ordenar): {len(data_list):,} filas.")


    # 3. DEPURACIÓN NIVEL 2 (QuickSort-style)
    print("\n[PASO 2] Depuración (QuickSort-style)...")
    start_depuracion = time.time()
    n_validos = depurar_y_particionar(data_list)
    arr_depurado = data_list[:n_validos]
    end_depuracion = time.time()
    print(f"   Tiempo de depuración: {end_depuracion - start_depuracion:.4f} segundos")
    print(f"   Registros finales a ordenar: {n_validos:,} filas.")
    print("-------------------------------------------------------")


    # 4. ORDENAMIENTO (Merge Sort)
    print("\n[PASO 3] Ordenamiento (Merge Sort)...")
    start_merge = time.time()
    datos_ordenados = merge_sort(arr_depurado)
    end_merge = time.time()
    print(f"   Tiempo de ordenamiento: {end_merge - start_merge:.4f} segundos")
    print("-------------------------------------------------------")


    # 5. ESCRITURA DE REPORTE
    print(f"\n[PASO 4] Generando archivo CSV: {archivo_salida}")
    start_escritura = time.time()
    try:
        # Solución de Codificación: Usar encoding='utf-8' de forma explícita
        with open(archivo_salida, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Anio', 'Ciudad', 'TemperaturaPromedio'])

            for row in datos_ordenados:
                ciudad, anio, temp = row
                writer.writerow([anio, f'"{ciudad}"', f'{temp:.2f}'])

    except Exception as e:
        print(f"\n ERROR: Al escribir el archivo. Asegúrese de que el sistema soporte UTF-8. Detalles: {e}")
        return
    end_escritura = time.time()
    print(f"   Tiempo de escritura: {end_escritura - start_escritura:.4f} segundos")


    tiempo_FINAL = time.time()
    print("\n=======================================================")
    print(f" PROCESO COMPLETADO. Tiempo total: {tiempo_FINAL - tiempo_INICIAL:.4f} segundos.")
    print("=======================================================")


if __name__ == '__main__':
    multiprocessing.freeze_support()
    main()


Iniciando Procesamiento Paralelo y Ordenamiento de Datos

[PASO 1] Leyendo archivo CSV...
   Tiempo de lectura: 0.8723 segundos
   Filas totales cargadas: 1,048,575 registros.
-------------------------------------------------------

[PROCESAMIENTO PARALELO] Usando 8 núcleos para 11 bloques.
-------------------------------------------------------
   Tiempo de limpieza paralelo: 2.3733 segundos
   Registros limpios (listos para ordenar): 992,090 filas.

[PASO 2] 🧹 Depuración (QuickSort-style)...
   Tiempo de depuración: 0.3238 segundos
   Registros finales a ordenar: 992,007 filas.
-------------------------------------------------------

[PASO 3] Ordenamiento (Merge Sort)...
   Tiempo de ordenamiento: 7.0409 segundos
-------------------------------------------------------

[PASO 4] Generando archivo CSV: procesamiento_paralelo_python_ordenado.csv
   Tiempo de escritura: 2.2952 segundos

 PROCESO COMPLETADO. Tiempo total: 13.5415 segundos.
